<h1>SQL Server 2019 Big Data Cluster</h1>


## Try to connect to the SQL Server master instance

In [1]:
select @@version, @@servername

(1 row affected)

Total execution time: 00:00:00.049

(No column name),(No column name)
Microsoft SQL Server 2019 (RTM-GDR) (KB4517790) - 15.0.2070.41 (X64) Oct 28 2019 19:56:59 Copyright (C) 2019 Microsoft Corporation Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>,master-0


## Create a demo database

In [2]:
CREATE DATABASE DemoDB;
GO

Commands completed successfully.

Total execution time: 00:00:10.118

## and an external datasource

In [3]:
USE DemoDB;
GO
CREATE EXTERNAL DATA SOURCE [SqlStoragePool]
    WITH (LOCATION = N'sqlhdfs://controller-svc:8080/default');

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.476

## Viewing the file : 6 header rows to exclude
## So, creating a file format accordingly

In [4]:
USE DemoDB;
GO
CREATE EXTERNAL FILE FORMAT [FileFormat_WxLog]
    WITH (FORMAT_TYPE = DELIMITEDTEXT, 
          FORMAT_OPTIONS (FIELD_TERMINATOR = N';', 
                          STRING_DELIMITER = N'\"', 
                          FIRST_ROW = 7));

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.188

## and then create the external table with a structure reflecting CSV structure file

In [5]:
USE DemoDB;
GO
CREATE EXTERNAL TABLE [dbo].[WxLog]
(
    [Date] [varchar](50) NOT NULL,
	[Time] [varchar](50) NOT NULL,
	[Baro] [varchar](50) NULL,
	[QNH] [varchar](50) NULL,
	[Gust Speed] [varchar](50) NULL,
	[Gust Dir] [varchar](50) NULL,
	[Avg Speed] [varchar](50) NULL,
	[Avg Dir] [varchar](50) NULL,
	[Rain Rate] [varchar](50) NULL,
	[Rain] [varchar](50) NULL,
	[UV] [varchar](50) NULL,
	[Temp 0] [varchar](50) NULL,
	[DewPt 0] [varchar](50) NULL,
	[RH 0] [varchar](50) NULL,
	[Temp 1] [varchar](50) NULL,
	[DewPt 1] [varchar](50) NULL,
	[RH 1] [varchar](50) NULL,
	[Temp 2] [varchar](50) NULL,
	[DewPt 2] [varchar](50) NULL,
	[RH 2] [varchar](50) NULL
)
WITH (LOCATION = N'/meteo/WxLog.csv', 
      DATA_SOURCE = [SqlStoragePool], 
	  FILE_FORMAT = [FileFormat_WxLog]);

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.362

## Now we can run some testing

In [6]:
USE DemoDB;
GO
SELECT TOP 10 * FROM [dbo].[WxLog]

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:04.093

Date,Time,Baro,QNH,Gust Speed,Gust Dir,Avg Speed,Avg Dir,Rain Rate,Rain,UV,Temp 0,DewPt 0,RH 0,Temp 1,DewPt 1,RH 1,Temp 2,DewPt 2,RH 2
11/05/2013,13:25,1024.00,1024.00,16.92,270,13.32,284,0.00,366.5,NULL,21.40,8.66,44,16.90,-1.72,28,19.10,19.10,100
11/05/2013,13:26,1024.00,1024.00,16.92,248,14.11,266,0.00,366.5,NULL,21.40,8.66,44,16.90,-1.72,28,19.20,19.20,100
11/05/2013,13:27,1024.00,1024.00,16.20,248,13.29,262,0.00,366.5,NULL,21.40,8.66,44,16.85,-1.76,28,19.20,19.20,100
11/05/2013,13:28,1024.00,1024.00,16.92,293,12.50,293,0.00,366.5,NULL,21.40,8.66,44,17.00,-1.63,28,19.30,19.30,100
11/05/2013,13:29,1024.00,1024.00,9.36,248,12.08,286,0.00,366.5,NULL,21.40,8.66,44,17.20,-1.46,28,19.30,19.30,100
11/05/2013,13:30,1024.00,1024.00,14.04,293,8.71,268,0.00,366.5,NULL,21.40,8.66,44,17.30,-1.37,28,NULL,NULL,
11/05/2013,13:31,1024.00,1024.00,9.72,293,9.00,293,0.00,366.5,NULL,21.40,8.66,44,17.40,-1.29,28,19.50,19.50,100
11/05/2013,13:32,1024.00,1024.00,12.60,293,9.79,288,0.00,366.5,NULL,21.40,8.66,44,17.60,-0.63,29,19.60,19.60,100
11/05/2013,13:33,1024.00,1024.00,12.24,293,9.43,266,0.00,366.5,NULL,21.40,8.66,44,17.50,-1.20,28,19.60,19.60,100
11/05/2013,13:34,1024.00,1024.00,12.60,270,9.62,266,0.00,366.5,NULL,21.40,8.66,44,17.85,-0.42,29,19.50,19.50,100


# We can switch to Spark engine to query the HDFS storage
## with the PySpark kernel



In [3]:
# Read the CSV file(s) into a spark dataframe and print schema
results = spark.read \
    .option("inferSchema", "true") \
    .csv('/csvfiles/temperature-last-year_poolhouse.csv') \
    .toDF("DateTime","Humidity","Temperature","Temperature_range (low)","Temperature_range (high)")
results.printSchema()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1574457022567_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DateTime: string (nullable = true)
 |-- Humidity: string (nullable = true)
 |-- Temperature: string (nullable = true)
 |-- Temperature_range (low): string (nullable = true)
 |-- Temperature_range (high): string (nullable = true)

In [4]:
results.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

367

In [5]:
results.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|Temperature_range (low)|Temperature_range (high)|
+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|   Temperature_range...|    Temperature_range...|
|2018-05-14 00:00:00|      80|      10.06|                    8.8|                    11.2|
|2018-05-15 00:00:00|      88|      11.83|                   10.5|                    13.6|
|2018-05-16 00:00:00|      83|      13.47|                   11.7|                    16.6|
|2018-05-17 00:00:00|      84|      14.69|                   12.9|                    18.1|
|2018-05-18 00:00:00|      82|      15.91|                   11.1|                    20.8|
|2018-05-19 00:00:00|      76|      17.69|                   13.9|                    21.6|
|2018-05-20 00:00:00|      67|      19.07|                   12.4|              

In [6]:
results.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|Temperature_range (low)|Temperature_range (high)|
+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|   Temperature_range...|    Temperature_range...|
|2018-05-14 00:00:00|      80|      10.06|                    8.8|                    11.2|
|2018-05-15 00:00:00|      88|      11.83|                   10.5|                    13.6|
|2018-05-16 00:00:00|      83|      13.47|                   11.7|                    16.6|
|2018-05-17 00:00:00|      84|      14.69|                   12.9|                    18.1|
+-------------------+--------+-----------+-----------------------+------------------------+
only showing top 5 rows

## We can filter the dataframe

In [7]:
results.filter("Humidity > 70").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|Temperature_range (low)|Temperature_range (high)|
+-------------------+--------+-----------+-----------------------+------------------------+
|2018-05-14 00:00:00|      80|      10.06|                    8.8|                    11.2|
|2018-05-15 00:00:00|      88|      11.83|                   10.5|                    13.6|
|2018-05-16 00:00:00|      83|      13.47|                   11.7|                    16.6|
|2018-05-17 00:00:00|      84|      14.69|                   12.9|                    18.1|
|2018-05-18 00:00:00|      82|      15.91|                   11.1|                    20.8|
|2018-05-19 00:00:00|      76|      17.69|                   13.9|                    21.6|
|2018-05-28 00:00:00|      82|      18.27|                   16.4|                    19.8|
|2018-05-29 00:00:00|      82|         19|                   16.4|              

In [8]:
results.filter("Humidity > 70").filter("Temperature > 15").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|Temperature_range (low)|Temperature_range (high)|
+-------------------+--------+-----------+-----------------------+------------------------+
|2018-05-19 00:00:00|      76|      17.69|                   13.9|                    21.6|
|2018-05-28 00:00:00|      82|      18.27|                   16.4|                    19.8|
|2018-05-29 00:00:00|      82|         19|                   16.4|                    22.2|
|2018-05-30 00:00:00|      84|      18.23|                   16.2|                    20.6|
|2018-05-31 00:00:00|      79|      18.97|                   15.7|                    23.4|
|2018-06-03 00:00:00|      73|      20.36|                   15.9|                    25.2|
|2018-06-04 00:00:00|      73|      20.77|                   15.5|                      26|
|2018-06-05 00:00:00|      79|      19.86|                   16.8|              

In [11]:
results.select("temperature","Humidity").summary().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+
|summary|       temperature|          Humidity|
+-------+------------------+------------------+
|  count|               367|               367|
|   mean|15.283779680952737| 66.59827489034282|
| stddev|7.2870462623946715|13.000718440852483|
|    min|             -0.09|  36.9686098654709|
|    25%|              9.49|              57.0|
|    50%|             14.17|              67.0|
|    75%|             21.64|              78.0|
|    max|       Temperature|          Humidity|
+-------+------------------+------------------+

## We can switch a TSQL like syntax to query the dataframe

In [12]:
results.select("temperature","Humidity").show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+
|temperature|Humidity|
+-----------+--------+
|Temperature|Humidity|
|      10.06|      80|
|      11.83|      88|
|      13.47|      83|
|      14.69|      84|
|      15.91|      82|
|      17.69|      76|
|      19.07|      67|
|      19.26|      65|
|      19.31|      69|
+-----------+--------+
only showing top 10 rows

# We can also use some real TSQL statements.
Let's creae a kind of view ands make some queries

In [13]:
results.createOrReplaceTempView("poolhouse")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
spark.sql("SELECT * from poolhouse").show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|Temperature_range (low)|Temperature_range (high)|
+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|   Temperature_range...|    Temperature_range...|
|2018-05-14 00:00:00|      80|      10.06|                    8.8|                    11.2|
|2018-05-15 00:00:00|      88|      11.83|                   10.5|                    13.6|
|2018-05-16 00:00:00|      83|      13.47|                   11.7|                    16.6|
|2018-05-17 00:00:00|      84|      14.69|                   12.9|                    18.1|
|2018-05-18 00:00:00|      82|      15.91|                   11.1|                    20.8|
|2018-05-19 00:00:00|      76|      17.69|                   13.9|                    21.6|
|2018-05-20 00:00:00|      67|      19.07|                   12.4|              

In [15]:
spark.sql("SELECT * from poolhouse LIMIT 10").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|Temperature_range (low)|Temperature_range (high)|
+-------------------+--------+-----------+-----------------------+------------------------+
|           DateTime|Humidity|Temperature|   Temperature_range...|    Temperature_range...|
|2018-05-14 00:00:00|      80|      10.06|                    8.8|                    11.2|
|2018-05-15 00:00:00|      88|      11.83|                   10.5|                    13.6|
|2018-05-16 00:00:00|      83|      13.47|                   11.7|                    16.6|
|2018-05-17 00:00:00|      84|      14.69|                   12.9|                    18.1|
|2018-05-18 00:00:00|      82|      15.91|                   11.1|                    20.8|
|2018-05-19 00:00:00|      76|      17.69|                   13.9|                    21.6|
|2018-05-20 00:00:00|      67|      19.07|                   12.4|              

In [16]:
spark.sql("SELECT MIN(Temperature),MAX(Temperature),AVG(Temperature) from poolhouse").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------------+--------------------------------+
|min(Temperature)|max(Temperature)|avg(CAST(Temperature AS DOUBLE))|
+----------------+----------------+--------------------------------+
|           -0.09|     Temperature|              15.283779680952737|
+----------------+----------------+--------------------------------+

In [17]:
spark.sql("SELECT DateTime,Temperature,LEAD(Temperature) OVER (order by DateTime) as NextValue,avg(Temperature) OVER () as avgTemp from poolhouse").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+---------+------------------+
|           DateTime|Temperature|NextValue|           avgTemp|
+-------------------+-----------+---------+------------------+
|2018-05-14 00:00:00|      10.06|    11.83|15.283779680952737|
|2018-05-15 00:00:00|      11.83|    13.47|15.283779680952737|
|2018-05-16 00:00:00|      13.47|    14.69|15.283779680952737|
|2018-05-17 00:00:00|      14.69|    15.91|15.283779680952737|
|2018-05-18 00:00:00|      15.91|    17.69|15.283779680952737|
|2018-05-19 00:00:00|      17.69|    19.07|15.283779680952737|
|2018-05-20 00:00:00|      19.07|    19.26|15.283779680952737|
|2018-05-21 00:00:00|      19.26|    19.31|15.283779680952737|
|2018-05-22 00:00:00|      19.31|    20.69|15.283779680952737|
|2018-05-23 00:00:00|      20.69|    21.14|15.283779680952737|
|2018-05-24 00:00:00|      21.14|    20.15|15.283779680952737|
|2018-05-25 00:00:00|      20.15|    21.54|15.283779680952737|
|2018-05-26 00:00:00|      21.54|    21.87|15.283779680

## We can also work on multiple files in the same folder

In [18]:
allfiles = spark.read \
    .option("inferSchema", "true") \
    .csv('/csvfiles/*.csv') \
    .toDF("DateTime","Humidity","Temperature","Temperature_range (low)","Temperature_range (high)")
allfiles.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1835

In [19]:
allfiles.select("temperature","Humidity").summary().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-----------------+
|summary|         temperature|         Humidity|
+-------+--------------------+-----------------+
|  count|                1835|             1835|
|   mean|  18.533133029743702|41.27881217307737|
| stddev|  6.2948649670237495|22.29592444791483|
|    min|               -0.09|            10.01|
|    25%|                14.3|            18.96|
|    50%|               21.45|             45.0|
|    75%|               22.97|             59.0|
|    max|Temperature_range...|      Temperature|
+-------+--------------------+-----------------+

## It is also possible to use the JOIN operator between dataframes

In [21]:
bathroom = spark.read \
    .option("inferSchema", "true") \
    .csv('/csvfiles/temperature-last-year_salledebain.csv') \
    .toDF("DateTime","Humidity","Temperature","Temperature_range (low)","Temperature_range (high)")

livingroom = spark.read \
    .option("inferSchema", "true") \
    .csv('/csvfiles/temperature-last-year_sejour.csv') \
    .toDF("DateTime","Humidity","Temperature","Temperature_range (low)","Temperature_range (high)")    

bathroom.select("DateTime","temperature","Humidity").join(livingroom.select("DateTime","temperature","Humidity"),"DateTime").show(10)  


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+--------+-----------+--------+
|           DateTime|temperature|Humidity|temperature|Humidity|
+-------------------+-----------+--------+-----------+--------+
|           DateTime|Temperature|Humidity|Temperature|Humidity|
|2018-05-14 00:00:00|      21.32|      57|      21.78|      48|
|2018-05-15 00:00:00|      21.27|      58|      21.61|      50|
|2018-05-16 00:00:00|      21.15|      61|      21.73|      52|
|2018-05-17 00:00:00|      21.14|      64|      22.14|      55|
|2018-05-18 00:00:00|      21.63|      67|      22.53|      56|
|2018-05-19 00:00:00|      21.83|      68|      22.39|      59|
|2018-05-20 00:00:00|      21.78|      64|      22.84|      57|
|2018-05-21 00:00:00|       22.1|      63|      22.94|      57|
|2018-05-22 00:00:00|      22.55|      68|      23.25|      58|
+-------------------+-----------+--------+-----------+--------+
only showing top 10 rows